In [9]:
import os
import json
import requests
import pandas as pd
from datetime import date, datetime
from typing import Any, Dict, List, Optional, Tuple
from dotenv import load_dotenv

In [10]:
load_dotenv()

FIREBASE_API_KEY   = os.getenv("GOOGLE_FIREBASE_API_KEY")
FIREBASE_EMAIL     = os.getenv("GOOGLE_FIREBASE_ACCOUNT_EMAIL")
FIREBASE_PASSWORD  = os.getenv("GOOGLE_FIREBASE_ACCOUNT_PASSWORD")
GOOGLE_PROJECT     = os.getenv("GOOGLE_PROJECT")
FIRESTORE_DATABASE = os.getenv("GOOGLE_FIRESTORE_DATABASE", "(default)")
FIRESTORE_COLLECT  = os.getenv("GOOGLE_FIRESTORE_COLLECTION")

In [11]:
def firebase_sign_in(api_key: str, email: str, password: str) -> str:
    """Return Firebase ID token from email/password."""
    url = f"https://identitytoolkit.googleapis.com/v1/accounts:signInWithPassword?key={api_key}"
    resp = requests.post(url, json={"email": email, "password": password, "returnSecureToken": True})
    resp.raise_for_status()
    data = resp.json()
    token = data.get("idToken")
    if not token:
        raise RuntimeError("Firebase sign-in succeeded but no idToken present.")
    return token

In [12]:
def decode_fs_value(v: Dict[str, Any]) -> Any:
    if "nullValue" in v: return None
    if "booleanValue" in v: return bool(v["booleanValue"])
    if "integerValue" in v: return int(v["integerValue"])
    if "doubleValue"  in v: return float(v["doubleValue"])
    if "timestampValue" in v: return v["timestampValue"]
    if "stringValue" in v: return v["stringValue"]
    if "mapValue" in v:
        fields = v["mapValue"].get("fields", {})
        return {k: decode_fs_value(fields[k]) for k in fields}
    if "arrayValue" in v:
        vals = v["arrayValue"].get("values", [])
        return [decode_fs_value(x) for x in vals]
    # bytesValue, referenceValue, geoPointValue not expected here
    return v

def decode_fs_document(doc: Dict[str, Any]) -> Tuple[str, Dict[str, Any]]:
    name = doc.get("name", "")
    fields = doc.get("fields", {})
    return name, {k: decode_fs_value(fields[k]) for k in fields}

In [13]:
def run_query_by_yyyymmdd(
    id_token: str,
    project: str,
    database: str,
    collection: str,
    start_yyyymmdd: int,
    end_yyyymmdd:   int,
) -> List[Dict[str, Any]]:
    """
    Uses structuredQuery to fetch documents in [start, end], ordered by yyyymmdd asc.
    Returns decoded documents (fields mapped to native Python).
    """
    url = f"https://firestore.googleapis.com/v1/projects/{project}/databases/{database}/documents:runQuery"
    headers = {"Authorization": f"Bearer {id_token}", "Accept": "application/json"}
    body = {
        "structuredQuery": {
            "from": [{"collectionId": collection}],
            "where": {
                "compositeFilter": {
                    "op": "AND",
                    "filters": [
                        {
                            "fieldFilter": {
                                "field": {"fieldPath": "yyyymmdd"},
                                "op": "GREATER_THAN_OR_EQUAL",
                                "value": {"integerValue": str(start_yyyymmdd)},
                            }
                        },
                        {
                            "fieldFilter": {
                                "field": {"fieldPath": "yyyymmdd"},
                                "op": "LESS_THAN_OR_EQUAL",
                                "value": {"integerValue": str(end_yyyymmdd)},
                            }
                        },
                    ],
                }
            },
            "orderBy": [{"field": {"fieldPath": "yyyymmdd"}, "direction": "ASCENDING"}],
        }
    }

    resp = requests.post(url, headers=headers, json=body)
    resp.raise_for_status()

    # runQuery returns a stream of objects with optional 'document'
    docs: List[Dict[str, Any]] = []
    for item in resp.json():
        doc = item.get("document")
        if not doc:
            continue
        _, decoded = decode_fs_document(doc)
        docs.append(decoded)
    return docs

In [14]:
def _extract_rows_for(
    day_fields: Dict[str, Any],
    prop: str,                # 'languages' | 'editors' | 'projects'
    day_date: date,
) -> Optional[pd.DataFrame]:
    arr = day_fields.get(prop) or []  # default empty
    if not isinstance(arr, list) or len(arr) == 0:
        return None

    # Each element: {"name": <str>, "total_seconds": <float>} per your writer
    rows = []
    for entry in arr:
        # be defensive if schema drifts
        name = (entry or {}).get("name")
        total_seconds = (entry or {}).get("total_seconds", 0.0) or 0.0
        if name is None:
            continue
        rows.append({"type": prop, "value": name, "total_seconds": float(total_seconds), "date": day_date})

    if not rows:
        return None
    return pd.DataFrame(rows)

def _yyyymmdd_to_date(n: int) -> date:
    s = str(n)
    return date(int(s[0:4]), int(s[4:6]), int(s[6:8]))

def read_firestore_into_df(
    start: Optional[date] = None,
    end:   Optional[date] = None,
) -> pd.DataFrame:
    """
    Fetch WakaTime-like stats from Firestore and return a tidy DataFrame with
    columns: ['type','value','total_seconds','date'] mirroring your local JSON loader.
    """
    id_token = firebase_sign_in(FIREBASE_API_KEY, FIREBASE_EMAIL, FIREBASE_PASSWORD)

    # Default to full range if not specified
    if start is None:
        start = date(2016, 10, 21)
    if end is None:
        end = date.today()

    start_yyyymmdd = int(start.strftime("%Y%m%d"))
    end_yyyymmdd   = int(end.strftime("%Y%m%d"))

    docs = run_query_by_yyyymmdd(
        id_token=id_token,
        project=GOOGLE_PROJECT,
        database=FIRESTORE_DATABASE,
        collection=FIRESTORE_COLLECT,
        start_yyyymmdd=start_yyyymmdd,
        end_yyyymmdd=end_yyyymmdd,
    )

    df_list: List[pd.DataFrame] = []
    for d in docs:
        # Prefer canonical date from fields; fall back to yyyymmdd
        if "year" in d and "month" in d and "day" in d:
            try:
                day_dt = date(int(d["year"]), int(d["month"]), int(d["day"]))
            except Exception:
                day_dt = _yyyymmdd_to_date(int(d["yyyymmdd"]))
        else:
            day_dt = _yyyymmdd_to_date(int(d["yyyymmdd"]))

        for prop in ("languages", "editors", "projects"):
            frame = _extract_rows_for(d, prop, day_dt)
            if frame is not None:
                df_list.append(frame)

        # Optionally, you can also capture grand_total_seconds per day if needed:
        # d.get("grand_total_seconds", 0.0)

    if not df_list:
        # Return an empty, schema-correct frame
        return pd.DataFrame(columns=["type", "value", "total_seconds", "date"])

    df = pd.concat(df_list, ignore_index=True)

    # Ensure expected dtypes and cleanups to mirror your original script
    df["total_seconds"] = pd.to_numeric(df["total_seconds"], errors="coerce").fillna(0.0)
    df["date"] = pd.to_datetime(df["date"]).dt.date
    # No 'color' column in Firestore input; nothing to drop here.

    return df

In [16]:
df = read_firestore_into_df()
df

,type,value,total_seconds,date
0,languages,Python,12983.749285,2016-10-22
1,languages,Other,8752.893072,2016-10-22
2,languages,PowerShell,2375.408296,2016-10-22
3,languages,Markdown,412.366297,2016-10-22
4,languages,Batchfile,284.513934,2016-10-22
...,...,...,...,...
27059,languages,Markdown,107.129000,2025-10-25
27060,languages,Other,91.884000,2025-10-25
27061,languages,Git Config,0.825000,2025-10-25
27062,editors,VS Code,2871.970000,2025-10-25


In [17]:
df.to_csv('summarized.csv')

df[df.type == 'languages'].to_csv('redacted.csv')